# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')[:100]
test_paths = glob('dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
#train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Checkpoint

In [14]:
save_path = 'checkpoints'

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path,
                                                monitor='val_acc', verbose=1,
                                                save_best_only=True, mode='max')

## Training

http://localhost:6006

In [16]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs, 
    callbacks=[checkpoint]
)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
3/3 [==============================] - 2s 590ms/step - loss: 2.3959 - accuracy: 0.0938 - val_loss: 2.3032 - val_accuracy: 0.0729
Epoch 2/5
3/3 [==============================] - 2s 593ms/step - loss: 2.3029 - accuracy: 0.1176 - val_loss: 2.3173 - val_accuracy: 0.0729
Epoch 3/5
3/3 [==============================] - 1s 456ms/step - loss: 2.2772 - accuracy: 0.1912 - val_loss: 2.3270 - val_accuracy: 0.0938
Epoch 4/5
3/3 [==============================] - 1s 417ms/step - loss: 2.2121 - accuracy: 0.1471 - val_loss: 2.3421 - val_accuracy: 0.0729
Epoch 5/5
3/3 [==============================] - 1s 496ms/step - loss: 2.3191 - accuracy: 0.0833 - val_loss: 2.3602 - val_accuracy: 0.0729


# Saving Model 
>### model 전체 저장
- model.save(경로)
- model = tf.keras.models.load_model(경로)
- 경로 명이 한글이 없도록!

In [17]:
save_path = 'my_model.h5'

In [18]:
model.save(save_path, include_optimizer=True)

In [19]:
model = tf.keras.models.load_model('my_model.h5')

# Saving Model - 2
>### model의 weight / architecture 저장
- model.save_weights
- model.to_json()

In [20]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [21]:
from tensorflow.keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [22]:
import h5py

model_file = h5py.File('my_model.h5','r+')    

In [23]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [24]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [29]:
model_file['model_weights']['conv2d']['conv2d']

<HDF5 group "/model_weights/conv2d/conv2d" (2 members)>

In [25]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [26]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [27]:
np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

array([[[[ 0.01715395, -0.03521417, -0.03214614,  0.10997819,
           0.12353923, -0.14016028,  0.04557806,  0.10228346,
           0.07807671, -0.05251225, -0.07918127, -0.09116083,
           0.02764348,  0.00752504, -0.02764323,  0.12461643,
           0.02656258,  0.01350196,  0.08069676, -0.13469988,
           0.02099181, -0.01686979,  0.07498396,  0.08448331,
           0.08602582, -0.11697201,  0.10425957,  0.11742344,
          -0.13077773, -0.06449135, -0.04169978,  0.0612827 ],
         [ 0.06138583, -0.04459596, -0.03095559, -0.03528496,
          -0.092962  , -0.02840158, -0.10226724,  0.07409155,
          -0.12176411,  0.11048147,  0.09223107,  0.02433428,
           0.12155504, -0.12297963,  0.08044858, -0.07843646,
           0.11223671,  0.06952491, -0.06878522, -0.07671657,
          -0.09500787, -0.03470901, -0.06764022, -0.10005303,
           0.12304599, -0.01058851, -0.0199094 ,  0.0276028 ,
          -0.11873493, -0.03026781,  0.01851539, -0.11644628],
      